# 向量嵌入

向量嵌入是将真实世界中复杂、高维的数据对象（如文本、图像、音频、视频等）转换为数学上易于处理的、低维、稠密的连续数值向量的技术。

这个向量的维度（长度）通常在几百到几千之间。

![](images/3_1_1.webp)

Embedding 的真正意义在于，它产生的向量不是随机数值的堆砌，而是对数据语义的数学编码。

在 Embedding 构建的向量空间中，语义上相似的对象，其对应的向量在空间中的距离会更近；而语义上不相关的对象，它们的向量距离会更远。

我们通常以余弦相似度、点积或者欧氏距离等指标来计算两个向量之间的相似度。

- 余弦相似度 (Cosine Similarity) ：计算两个向量夹角的余弦值。值越接近 1，代表方向越一致，语义越相似。这是最常用的度量方式。

- 点积 (Dot Product) ：计算两个向量的乘积和。在**向量归一化后**，点积等价于余弦相似度。

- 欧氏距离 (Euclidean Distance) ：计算两个向量在空间中的直线距离。距离越小，语义越相似。

## Embedding 在 RAG 中的作用

### 语义检索的基础

- 离线索引构建：将知识库内文档切分后，使用 Embedding 模型将每个文档块（Chunk）转换为向量，存入专门的向量数据库中。

- 在线查询检索：当用户提出问题时，使用**同一个 Embedding** 模型将用户的问题也转换为一个向量。

- 相似度计算：在向量数据库中，计算“问题向量”与所有“文档块向量”的相似度

- 召回上下文：选取相似度最高的 Top-K 个文档块，作为补充的上下文信息，与原始问题一同送给大语言模型（LLM）生成最终答案。

### 决定检索质量的关键

Embedding 的质量直接决定了 RAG 检索召回内容的准确性与相关性。

一个优秀的 Embedding 模型能够精准捕捉问题和文档之间的深层语义联系，即使用户的提问和原文的表述不完全一致。

## Embedding 技术发展

### 静态词嵌入：上下文无关的表示

为词汇表中的每个单词生成一个固定的、与上下文无关的向量。

例如，Word2Vec 通过 Skip-gram 和 CBOW 架构，利用局部上下文窗口学习词向量，并验证了向量运算的语义能力（如 国王 - 男人 + 女人 ≈ 王后）。

这种方法无法处理一词多义问题。在“苹果公司发布了新手机”和“我吃了一个苹果”中，“苹果”的词向量是完全相同的，这限制了其在复杂语境下的语义表达能力。

### 动态上下文嵌入

Transformer 架构的诞生带来了自注意力机制（Self-Attention），它允许模型在生成一个词的向量时，动态地考虑句子中所有其他词的影响。

BERT 模型利用 Transformer 的编码器，通过掩码语言模型（MLM）等自监督任务进行预训练，生成了深度上下文相关的嵌入。

同一个词在不同语境中会生成不同的向量，这有效解决了静态嵌入的一词多义难题。

###  RAG 对嵌入技术的新要求

- 领域自适应能力：嵌入模型具备领域自适应的能力，能够通过微调或使用指令（如 INSTRUCTOR 模型）来适应特定领域的术语和语义。

- 多粒度与多模态支持：RAG 系统需要处理的不仅仅是短句，还可能包括长文档、代码，甚至是图像和表格。这要求嵌入模型能够处理不同长度和类型的输入数据。

- 检索效率与混合检索：嵌入向量的维度和模型大小直接影响存储成本和检索速度。同时，为了结合语义相似性（密集检索）和关键词匹配（稀疏检索）的优点，支持混合检索的嵌入模型（如 BGE-M3）应运而生，在某些任务中成为提升召回率的关键。

## 嵌入模型训练原理

现代嵌入模型的核心通常是 Transformer 的编码器（Encoder）部分，通过堆叠多个 Transformer Encoder 层来构建一个深度的双向表示学习网络。

### 主要训练任务

**任务一：掩码语言模型 (Masked Language Model, MLM)**

**过程：**

随机地将输入句子中 15% 的词元（Token）替换为一个特殊的 [MASK] 标记。

让模型去预测这些被遮盖住的原始词元是什么。

**目标：**

通过这个任务，模型被迫学习每个词元与其上下文之间的关系，从而掌握深层次的语境语义

**任务二：下一句预测 (Next Sentence Prediction, NSP)**

**过程：**
构造训练样本，每个样本包含两个句子 A 和 B。

其中 50% 的样本，B 是 A 的真实下一句（IsNext）；另外 50% 的样本，B 是从语料库中随机抽取的句子（NotNext）。

让模型判断 B 是否是 A 的下一句。

**目标：**

这个任务让模型学习句子与句子之间的逻辑关系、连贯性和主题相关性。

**重要说明：**

后续的研究（如 RoBERTa）发现2，NSP 任务可能过于简单，甚至会损害模型性能。因此，许多现代的预训练模型（如 RoBERTa、SBERT）已经放弃了 NSP 任务。

## 效果增强策略

MLM 和 NSP 赋予了模型强大的基础语义理解能力，但为了在检索任务中表现更佳，现代嵌入模型通常会引入更具针对性的训练策略。

**度量学习 (Metric Learning)**

**思想：**

直接以“相似度”作为优化目标。

**方法：**

收集大量相关的文本对（例如，（问题，答案）、（新闻标题，正文））。

训练的目标是优化向量空间中的相对距离：让“正例对”的向量表示在空间中被“拉近”，而“负例对”的向量表示被“推远”。

关键在于**优化排序关系**，而非追求绝对的相似度值（如 1 或 0），因为过度追求极端值可能导致模型过拟合。

**对比学习 (Contrastive Learning)**

**思想：**

在向量空间中，将相似的样本“拉近”，将不相似的样本“推远”。

**方法：**

构建一个三元组（Anchor, Positive, Negative）。

其中，Anchor 和 Positive 是相关的（例如，同一个问题的两种不同问法），Anchor 和 Negative 是不相关的。

训练的目标是让 distance(Anchor, Positive) 尽可能小，同时让 distance(Anchor, Negative) 尽可能大。